In [33]:
import pandas as pd
import awswrangler as awr
import openpyxl
import os
import shutil
import unicodedata
import win32com.client as win32

#-----------------EXTRACT
path_to_query = r"C:\Users\raphael.almeida\Documents\Projetos\Seguro de Vida Motorista\listagem_mestra_ajustada.sql"

with open (path_to_query,'r') as reading:
    query_readed = reading.read()

df = awr.athena.read_sql_query(query_readed,'silver')

In [34]:
#-----------------TRANSFORM
#adding empty e-mails with the e-mail common pattern
row = ((df['e_mail'] == "") | (df['e_mail'].isnull())) & df['unidade'].str.contains('UNIDADE ')
df.loc[row, 'e_mail'] = 'unidade.' + df.loc[row, 'unidade'].str.replace('UNIDADE ', '').str.lower() + '@transdesk.com.br'


In [35]:
#removing empty spaces
df['e_mail'] = df['e_mail'].str.replace(' ','')

#removing diacritics with unicodedata
def remove_diacritics (texto):
    if isinstance(texto,str):
        return ''.join(c for c in unicodedata.normalize('NFKD',texto) if not unicodedata.combining(c))
    return texto

df['e_mail'] = df['e_mail'].apply(remove_diacritics)

df = df.reset_index(drop=True)


In [36]:
len(df)

8828

base_analise

In [ ]:

#excluir coluna de e-mail pois está diferenciando na análise
#df = df.drop('e_mail', axis=1)
df = df.drop_duplicates()

#pegar apenas os conjuntos obrigatorios de seguro de vida
beneficios_conjuntos = df.groupby(['conjunto', 'cooperativa'])['benefits_id'].unique()
conjuntos_validos = beneficios_conjuntos[(beneficios_conjuntos.apply(lambda x: {2,6}.issubset((x))))|
                                         (beneficios_conjuntos.apply(lambda x: {24,28}.issubset((x))))|
                                         (beneficios_conjuntos.apply(lambda x: {40,44}.issubset((x))))].index

df_obrigatorio = df[df.set_index(['conjunto','cooperativa']).index.isin(conjuntos_validos)].copy()

#criar colunas para posterior agregação
df_obrigatorio['casco'] = df_obrigatorio['benefits_id'].apply(lambda x: x if (x == 2 or x == 24 or x == 40) else pd.NA)
df_obrigatorio['seg_vida'] = df_obrigatorio['benefits_id'].apply(lambda x: x if (x==6 or x==28 or x==44) else pd.NA)

#filtrando apenas os que fazem sentido para a análise (casco e seguro de vida)
df_obrigatorio_filtrado = df_obrigatorio[df_obrigatorio['benefits_id'].isin([2,6,24,28,40,44])]

In [37]:
df_obrigatorio_filtrado.head()

,cooperativa,unidade,cliente,matricula,conjunto,coverage,beneficio,motorista,id_placa,placa,...,benefits_id,categoria,tipo_categoria,faixa_valor,tabela_ativa,tipo_veiculo,tipo,e_mail,casco,seg_vida


conjuntos_analise

In [17]:


df_analise = df_obrigatorio_filtrado.groupby(['cooperativa','unidade','matricula','conjunto','e_mail']).agg(
    qtd_seg_vida = ('seg_vida', 'count'),
    qtd_casco = ('casco', 'count'),
    qtd_motorista = ('motorista', 'count')
).reset_index()

#df_analise.drop_duplicates('e_mail', keep='first')
df_analise.sort_values(by='conjunto', ascending=False)

df_analysis=df_analise[(df_analise['qtd_motorista']!=df_analise['qtd_casco'])|
                      (df_analise['qtd_motorista']!=df_analise['qtd_seg_vida'])|
                      (df_analise['qtd_casco']!=df_analise['qtd_seg_vida'])]

df_analysis2 = df_analysis[df_analysis['qtd_seg_vida']<df_analysis['qtd_casco']]
df_analysis2.head()
#df_analysis = df_analise[row, ['cooperativa','unidade','matricula','conjunto','qtd_motoristas','qtd_cascos','qtd_segvida']]

file_path  = r'C:\Users\raphael.almeida\Documents\Projetos\Seguro de Vida Motorista\conjuntos_analise.xlsx'
df_analysis2.to_excel(file_path, engine = 'openpyxl', index = False)


parametros de e-mail

In [20]:
#-----------------LOAD
#Function to send e-mails

def enviar_email(p_email, p_unidade, p_empresa, tabela_html):    
    outlook = win32.Dispatch('outlook.application')  #integrating python with outlook 

    #creating e-mail variable to use
    email = outlook.CreateItem(0)

    #configuring the e-mail - ALTEREI PARA TESTE
    email.To = p_email
    email.Subject = "[PENDÊNCIA DE SEGURO DE VIDA - MOTORISTAS] - Análise de Dados"
    email.HTMLBody = f"""
    <p>Prezado(a),<p/>

    <p>A {p_unidade} ({p_empresa}) possui pendências de cadastramento de seguro de vida para motoristas.<p/>

    <p>A quantidade de coberturas de cavalo é maior do que a quantidade de coberturas de seguro de vida (motoristas).<p/>

    <p>{tabela_html}<p/>

    <p>Favor, cadastrar os seguros de vida remanescentes para cômputo na base de dados da empresa, obrigado!<p/>

    <p>Atenciosamente,<p/>

    <p>Equipe Análise de Dados - Grupo Unus<p/>

    <p>(<i>Este é um e-mail automático, por favor não responda<i/>)<p/>
 
    """

    email.Send()

    print("Email enviado")

função envio email real

In [ ]:
df_analysis3 = df_analysis2.drop('qtd_motorista', axis=1)
df_email = df_analysis3[['cooperativa','unidade','e_mail']].drop_duplicates('e_mail', keep='first')

for _, row in df_email.iterrows():
    iEmail = row['e_mail']
    iUnidade = row['unidade']
    iCooperativa = row['cooperativa']

    df_filtrado = df_analysis3[(df_analysis3['unidade']==iUnidade) & (df_analysis3['cooperativa']==iCooperativa)]

    if df_filtrado.empty:
        print(f"Nenhum dado encontrado para {iUnidade} - {iCooperativa}. E-mail não enviado.")
        continue  

    df_tabela = df_filtrado.to_html(index=False, border=1)

    # if __name__ == '__main__':
    #     enviar_email(iEmail, iUnidade, iCooperativa, df_tabela)

    print(f"E-mail enviado para {iEmail}: {iUnidade}({iCooperativa})")



função envio e-mail teste

In [23]:
df_analysis3 = df_analysis2.drop('qtd_motorista', axis=1)
df_analysis3['e_mail']='dados13@grupounus.com.br'
df_email = df_analysis3[['cooperativa','unidade','e_mail']].drop_duplicates('unidade', keep='first')

for _, row in df_email.iterrows():
    iEmail = row['e_mail']
    iUnidade = row['unidade']
    iCooperativa = row['cooperativa']

    df_filtrado = df_analysis3[(df_analysis3['unidade']==iUnidade) & (df_analysis3['cooperativa']==iCooperativa)]

    if df_filtrado.empty:
        print(f"Nenhum dado encontrado para {iUnidade} - {iCooperativa}. E-mail não enviado.")
        continue  

    df_tabela = df_filtrado.to_html(index=False, border=1, classes = "tabela")

    df_tabela = f"""
    <style>
        .tabela {{
            width: 100%;
            border-collapse: collapse;
        }}
        .tabela th, .tabela td {{
            text-align: center;  
            padding: 8px;
            border: 1px solid black;
        }}
    </style>
    {df_tabela}
    """

    if __name__ == '__main__':
        enviar_email(iEmail, iUnidade, iCooperativa, df_tabela)

    print(f"E-mail enviado para {iEmail}: {iUnidade}({iCooperativa})")


Email enviado
E-mail enviado para dados13@grupounus.com.br: UNIDADE CASCAVEL(Stcoop)
Email enviado
E-mail enviado para dados13@grupounus.com.br: UNIDADE CAMPO GRANDE(Viavante)
Email enviado
E-mail enviado para dados13@grupounus.com.br: UNIDADE CURITIBA(Viavante)
Email enviado
E-mail enviado para dados13@grupounus.com.br: UNIDADE DOURADOS(Viavante)
Email enviado
E-mail enviado para dados13@grupounus.com.br: UNIDADE FEIRA DE SANTANA(Viavante)
Email enviado
E-mail enviado para dados13@grupounus.com.br: UNIDADE ITAPEJARA(Viavante)
Email enviado
E-mail enviado para dados13@grupounus.com.br: UNIDADE JOINVILLE(Viavante)
Email enviado
E-mail enviado para dados13@grupounus.com.br: UNIDADE LONDRINA(Viavante)
Email enviado
E-mail enviado para dados13@grupounus.com.br: UNIDADE MARINGÁ(Viavante)


conjuntos_aberto

In [43]:
df_obrigatorio_filtrado = df_obrigatorio[df_obrigatorio['benefits_id'].isin([2,6,24,28,40,44])]

df_open = df_obrigatorio_filtrado.sort_values(by='conjunto', ascending=False)
#df_open = df_open[['cooperativa','unidade', 'cliente', 'matricula','conjunto','coverage','beneficio', 'motorista', 'placa', 'casco', 'seg_vida', 'tipo_veiculo', 'benefits_id']] 
file_path  = r'C:\Users\raphael.almeida\Documents\Projetos\Seguro de Vida Motorista\conjuntos_aberto.xlsx'
df_open.to_excel(file_path, engine = 'openpyxl', index = False)

In [ ]:


path_to_save = r'C:\Users\raphael.almeida\Documents\Projetos\Seguro de Vida Motorista'
file_name = 'Cobertura de Seguros Motoristas.xlsx'
final_path = os.path.join(path_to_save, file_name)
os.remove(final_path)
df.to_excel(final_path, engine='openpyxl', index = False)